In [ ]:
cd ../../../
./aestas.ps1 build

In [ ]:
#load "privateVar.fsx"
// use a individual file to store your private data, like:
// #r "C:\\Users\\YourUserName\\Source\\Aestas\\bin\\Debug\\net8.0\\aestas.dll"
// let geminiKey = "Your Api Key"
// let name = "Bot Name"
// let systemInstruction = "Your System Instruction"
open PrivateVar
open Aestas
open Aestas.Prim
open Aestas.Core
open Aestas.Core.BotHelper
open Aestas.AutoInit
open Aestas.Llms.Gemini

In [ ]:
// seq {InitTypes.Command} |> set |> AutoInit.init true
let model = GeminiLlm({
    api_key = Some geminiKey
    gcloudpath = None
    safetySettings = [|
        {category = "HARM_CATEGORY_HARASSMENT"; threshold = "BLOCK_NONE"}
        {category = "HARM_CATEGORY_HATE_SPEECH"; threshold = "BLOCK_NONE"}
        {category = "HARM_CATEGORY_SEXUALLY_EXPLICIT"; threshold = "BLOCK_NONE"}
        {category = "HARM_CATEGORY_DANGEROUS_CONTENT"; threshold = "BLOCK_NONE"}
    |]
    generation_configs = {
        temperature = 1.02
        maxLength = 4096
        topK = 64
        topP = 1.
    } |> Some
    }, 
    "gemini-1.5-flash-exp-0827")
let bot = 
    createBot {
        name = name
        model = model
        systemInstruction = systemInstruction |> Some
        systemInstructionBuilder = Builtin.buildSystemInstruction |> PipeLineChain |> Some
        contentLoadStrategy = None
        contentParseStrategy = None
        messageReplyStrategy = StrategyReplyAll |> Some
        messageCacheStrategy = None
        contextStrategy = None
        inputPrefixBuilder = Builtin.buildPrefix |> Some
        userCommandPrivilege = [
            0u, CommandPrivilege.High
        ] |> Some
    }
let mutable display = ""
let virtualDomain = VirtualDomain((fun s -> display <- s.ToString()), ignore, {
        uid = 1u
        name = name
    },{
        uid = 0u
        name = "User"
    }, 0u, "Test", true)
bindDomain bot virtualDomain

In [ ]:
open Aestas.Core.CommandHelper
let args = [Identifier "name"; String "Vespera"; Identifier "books"; AtomTuple [String "Math"; String "Physics"]]
let args', errors = args |> parseArguments [
    ParamString("name", None)
    ParamTuple("books", None)
    ParamNumber("age", None)
    ParamString("city", Some "Celestia")
]
args'["name"], args'["books"], args'["age"], args'["city"], errors

In [ ]:
type TestCommand() =
    interface ICommand with
        member _.Name = "test"
        member _.Help = "Test Command"
        member _.AccessibleDomain = CommandAccessibleDomain.All
        member _.Privilege = CommandPrivilege.Normal
        member _.Execute env args =
            env.log "02"; Unit

In [ ]:
TestCommand() |> addCommand bot

In [ ]:
TestCommand() |> updateCommand bot

In [ ]:
virtualDomain.Input([AestasText "你好"]) |> Async.Ignore |> Async.RunSynchronously
printfn "%s" display
virtualDomain.Messages |> Seq.iter (fun x -> let y = x.Parse() in printfn "%s: %s" y.sender.name x.Preview)

In [ ]:
bot.ClearCachedContext virtualDomain

In [ ]:
Logger.getLogs(0) |> Seq.iter (fun x -> printfn "%A" x)